In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

# Parameters
img_size = (150, 150)
batch_size = 32
epochs = 10

# Base dataset directory
base_dir = r'D:\DeepLearning_Exam_Datasets\forest_fire'
train_val_dir = os.path.join(base_dir, 'Training and Validation')
test_dir = os.path.join(base_dir, 'Testing')

# ImageDataGenerators with validation split
train_val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Training and validation generators
train_generator = train_val_datagen.flow_from_directory(
    train_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = train_val_datagen.flow_from_directory(
    train_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True,
    seed=42
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Model creation function
def create_model():
    model = Sequential([
        Flatten(input_shape=(150, 150, 3)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Train with Adam optimizer
model_adam = create_model()
model_adam.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_adam = model_adam.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=2
)

# Evaluate Adam on test set
test_loss_adam, test_acc_adam = model_adam.evaluate(test_generator)
print(f"\nAdam Test Accuracy: {test_acc_adam:.4f}")

# Predict & classification report Adam
y_pred_prob_adam = model_adam.predict(test_generator)
y_pred_adam = (y_pred_prob_adam > 0.5).astype(int).flatten()
y_true = test_generator.classes
print("\nAdam Classification Report:")
print(classification_report(y_true, y_pred_adam, target_names=list(test_generator.class_indices.keys())))

# Train with SGD optimizer
model_sgd = create_model()
model_sgd.compile(
    optimizer=SGD(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_sgd = model_sgd.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=2
)

# Evaluate SGD on test set
test_loss_sgd, test_acc_sgd = model_sgd.evaluate(test_generator)
print(f"\nSGD Test Accuracy: {test_acc_sgd:.4f}")

# Predict & classification report SGD
y_pred_prob_sgd = model_sgd.predict(test_generator)
y_pred_sgd = (y_pred_prob_sgd > 0.5).astype(int).flatten()
print("\nSGD Classification Report:")
print(classification_report(y_true, y_pred_sgd, target_names=list(test_generator.class_indices.keys())))

# Plot accuracy comparison
plt.plot(history_adam.history['accuracy'], label='Adam - Train Acc')
plt.plot(history_adam.history['val_accuracy'], label='Adam - Val Acc')
plt.plot(history_sgd.history['accuracy'], label='SGD - Train Acc')
plt.plot(history_sgd.history['val_accuracy'], label='SGD - Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy: Adam vs SGD')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
